In [17]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos, e, atan

In [18]:
# B,G,R
def compute_Hue(B, G, R):
    # prevent divide by 0, adding 0.00000001
    angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B) + 0.00000001)
    if B<= G:
        return acos(angle)
    elif B> G:
        return (2*pi- acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))

In [19]:
def convolution(F, F_length, src, height, width):
    
    I_ori = np.zeros((height, width))
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I_ori[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #margin = int(F_length/2)
 
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(height):
        for j in range(width):
            
            gsum = 0
            for m in range(0,F_length):
                idxY = i-F_length+m
                if idxY < 0 : idxY = -1*idxY
                elif idxY>=height : idxY = 2*height-idxY-1
                
                for n in range(0,F_length):
                    idxX = j-F_length+n
                    if idxX < 0 : idxX = -1*idxX
                    elif idxX>=width : idxX = 2*width-idxX-1
                    
                    gsum += F[m][n] * I_ori[idxY][idxX]
                    
                
            if gsum < 0:
                gsum = 0
            elif gsum > 255:
                gsum = 255
                    
            I[i][j] = gsum
            
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < F_length or i > height-F_length or j < F_length or j > width-F_length :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I_ori[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I_ori[i][j])
                    hsi2G_mean[i][j] = ((I_ori[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I_ori[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I_ori[i][j])
                    hsi2B_mean[i][j] = ((I_ori[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I_ori[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I_ori[i][j])
                    hsi2R_mean[i][j] = ((I_ori[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            
             
            
            temp_I = 0
            for m in range(F_length):
                for n in range(F_length):
                    temp_I = F[m][n] * I[i+m][j+n]
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2R_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2G_mean[i][j] += ((temp_I*3. - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2G_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2B_mean[i][j] += ((temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n]+2*pi/3
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2B_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2R_mean[i][j] += ((temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n] + 4*pi/3
    
    
    
    
    
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    
            
    return plt

In [32]:
src = cv2.imread('./test image/cameraman.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]

prewitt = np.zeros((height, width,3), dtype=np.uint8)
sobel = np.zeros((height, width,3), dtype=np.uint8)

Px = np.zeros((3, 3), dtype=np.float32)
Py = np.zeros((3, 3), dtype=np.float32)
Sx = np.zeros((3, 3), dtype=np.float32)
Sy = np.zeros((3, 3), dtype=np.float32)

for i in range(0, 3):
    Px[i][0] = -1
    Px[i][2] = 1
    Py[0][i] = 1
    Py[2][i] = -1
    
    Sx[i][0] = -1
    Sx[i][2] = 1
    Sy[0][i] = 1
    Sy[2][i] = -1
    
    Sx[1][i] *= 2
    Sy[i][1] *= 2

        
prewitt = (convolution(Sx, 3, src, height, width)**2 + convolution(Sy, 3, src, height, width)**2)**1/2
#prewitt = threshold(prewitt, height, width)
sobel = (convolution(Sx, 3, src, height, width)**2 + convolution(Sy, 3, src, height, width)**2)**1/2
#sobel = threshold(sobel, height, width)
#gaussian = gaussianfilter(src, height, width, maskheight, maskwidth)
'''
for i in range(6):
    median = medianfilter(src, height, width, maskheight, maskwidth)
    cv2.imshow("Median Filter "+str(i+1), median)
'''
cv2.imshow("Prewitt Filter", prewitt)
cv2.imshow("Sobel Filter", sobel)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
def threshold(img, height, width):
    
    result = np.zeros((height, width,3), dtype=np.uint8)

    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))
    hsi2B_negative = np.zeros((height, width))
    hsi2G_negative = np.zeros((height, width))
    hsi2R_negative = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]



    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

            if I[i][j] < 128:
                hsi2B_negative[i][j], hsi2G_negative[i][j], hsi2R_negative[i][j] = 0, 0, 0
            elif I[i][j]>=128:
                hsi2B_negative[i][j], hsi2G_negative[i][j], hsi2R_negative[i][j] = 255, 255, 255
            '''
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                H[i][j] = H[i][j]+4*pi/3        
            '''
    result = cv2.merge((hsi2B_negative,hsi2G_negative,hsi2R_negative))

    return result

In [ ]:
def PrewittX(src, height, width):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    Sx = np.zeros((3, 3), dtype=np.float32)
    
    for i in range(0, 3):
        Sx[i][0] = 1
        Sx[i][1] = 0
        Sx[i][2] = -1

    return Sx

In [ ]:
def PrewittX(src, height, width):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    Sy = np.zeros((3, 3), dtype=np.float32)
    
    for i in range(0, 3):
        Sy[0][i] = 1
        Sy[1][i] = 0
        Sy[2][i] = -1
        
    
    return Sy

In [97]:
def Prewitt(src, height, width):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #meanfilter는 mask라 사이즈 지정 잘 해주기
    Sx = np.zeros((3, 3), dtype=np.float32)
    Sy = np.zeros((3, 3), dtype=np.float32)
    Gx = np.zeros((3, 3), dtype=np.float32)
    Gy = np.zeros((3, 3), dtype=np.float32)
 
    for i in range(0, 3):
        Sx[i][0] = 1
        Sx[i][1] = 0
        Sx[i][2] = -1
        Sy[0][i] = 1
        Sy[1][i] = 0
        Sy[2][i] = -1

    print(Sx)
    print(Sy)
    
    margin = int(3/2)
 
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(height):
        for j in range(width):
            #if i < 3 or i > height-3 or j < 3 or j > width-3 :
            #    plt[i][j] = src[i][j]
            #    continue
            
            gsum = 0
            for m in range(0,3):
                idxY = i-margin+m
                if idxY < 0 : idxY = -1*idxY
                elif idxY>=height : idxY = 2*height-idxY-1
                
                for n in range(0,3):
                    idxX = i-margin+n
                    if idxX < 0 : idxX = -1*idxX
                    elif idxX>=width : idxX = 2*width-idxX-1
                    
                    gsum += Sx[m][n] * src[idxY][idxX]
                    
            #if gsum.any() < 0:
            #    gsum = 0
            #elif gsum.any() > 255:
            #    gsum = 255
                    
            plt[i][j] = gsum
            
    return plt

In [27]:
def Prewitt(src, height, width):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #meanfilter는 mask라 사이즈 지정 잘 해주기
    Sx = np.zeros((3, 3), dtype=np.float32)
    Sy = np.zeros((3, 3), dtype=np.float32)
    Gx = np.zeros((3, 3), dtype=np.float32)
    Gy = np.zeros((3, 3), dtype=np.float32)
 
    for i in range(0, 3):
        Sx[i][0] = 1
        Sx[i][1] = 0
        Sx[i][2] = -1
        Sy[0][i] = 1
        Sy[1][i] = 0
        Sy[2][i] = -1

    print(Sx)
    print(Sy)
        
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(height):
        for j in range(width):
            if i < 3 or i > height-3 or j < 3 or j > width-3 :
                plt[i][j] = src[i][j]
                continue
            
            gxtemp, gytemp, gsum = 0, 0, 0
            for m in range(0,3):
                for n in range(0,3):            
                    gxtemp += Sx[m][n] * src[i+m][j+n]
                    gytemp += Sy[m][n] * src[i+m][j+n]
            plt[i][j] = int(sqrt(gxtemp**2 + gytemp**2))
            #if gsum > 255:
            #    gsum = 255
            #if gsum < 0:
            #    gsum = 0

            
    return plt

In [47]:
def Prewitt(src, height, width):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #meanfilter는 mask라 사이즈 지정 잘 해주기
    Sx = np.zeros((3, 3), dtype=np.float32)
    Sy = np.zeros((3, 3), dtype=np.float32)
    
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(0, 3):
        Sx[i][0] = 1
        Sx[i][1] = 0
        Sx[i][2] = -1
        Sy[0][i] = 1
        Sy[1][i] = 0
        Sy[2][i] = -1
    
    for i in range(height):
        for j in range(width):
            if i < 3 or i > height-3 or j < 3 or j > width-3 :
                plt[i][j] = src[i][j]
                continue
            
            G_magnitude, G_direction = 0, 0
            #Gx = Sx[m][n] * src[i+m][j+n]
            #Gy = Sy[m][n] * src[i+m][j+n]
            for m in range(0,3):
                for n in range(0,3):            
                    G_magnitude = sqrt(abs(Sx[m][n] * src[i+m][j+n])**2 + abs(Sy[m][n] * src[i+m][j+n])**2)
                    G_direction = atan(Gy[m][n]/Gx[m][n])
            plt[i][j] = G_magnitude
            
    return plt

In [43]:
def Sobel(src, height, width, maskheight, maskwidth):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #meanfilter는 mask라 사이즈 지정 잘 해주기
    mean = np.zeros((maskheight, maskwidth), dtype=np.float32)
    
    pixel_num = maskheight*maskwidth
    
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            mean[i][j] = 1/pixel_num
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp += mean[m][n] * src[i+m][j+n]
            plt[i][j] = temp
            
    return plt